# SRA Roll-up

In [1]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [14]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

with open(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\sql_queries\population_sra_rollup.sql', 'r') as sql_file:
    sql_query = sql_file.read()

df =  pd.read_sql_query(sql_query, conn)
df

,sra,yr_id,long_name,population
0,Central San Diego,2010,Household Population,96
1,Central San Diego,2010,Group Quarters - Military,0
2,Central San Diego,2010,Group Quarters - College,0
3,Central San Diego,2010,Group Quarters - Other,0
4,Central San Diego,2010,Household Population,72
...,...,...,...,...
1311259,Southeastern San Diego,2011,Group Quarters - Other,0
1311260,Southeastern San Diego,2011,Household Population,375
1311261,Southeastern San Diego,2011,Group Quarters - Military,0
1311262,Southeastern San Diego,2011,Group Quarters - College,0


In [15]:
df = df.groupby(['sra', 'yr_id', 'long_name']).sum().reset_index()
df

,sra,yr_id,long_name,population
0,Alpine,2010,Group Quarters - College,0
1,Alpine,2010,Group Quarters - Military,0
2,Alpine,2010,Group Quarters - Other,138
3,Alpine,2010,Household Population,15135
4,Alpine,2011,Group Quarters - College,0
...,...,...,...,...
1963,Vista,2020,Household Population,104649
1964,Vista,2021,Group Quarters - College,0
1965,Vista,2021,Group Quarters - Military,0
1966,Vista,2021,Group Quarters - Other,2134


In [16]:
output = pd.pivot(df, index=(['sra', 'yr_id']), columns=['long_name'], values='population')
output['total_pop'] = output.sum(axis=1)
output['GQ_pop'] = output['Group Quarters - College'] + output['Group Quarters - Military'] + output['Group Quarters - Other']
output.columns.name = ''

output

Group Quarters - College  Group Quarters - Military  \
sra    yr_id                                                        
Alpine 2010                          0                          0   
       2011                          0                          0   
       2012                          0                          0   
       2013                          0                          0   
       2014                          0                          0   
...                                ...                        ...   
Vista  2017                          0                          0   
       2018                          0                          0   
       2019                          0                          0   
       2020                          0                          0   
       2021                          0                          0   

              Group Quarters - Other  Household Population  total_pop  GQ_pop  
sra    yr_id                                                                   
Alpine 2010                      138                 15135      15273     138  
       2011                      103                 15406      15509     103  
       2012                      105                 15599      15704     105  
       2013                       88                 15794      15882      88  
       2014                      119                 15872      15991     119  
...                              ...                   ...        ...     ...  
Vista  2017                     2271                104503     106774    2271  
       2018                     2359                104642     107001    2359  
       2019                     2344                104568     106912    2344  
       2020                     2360                104649     107009    2360  
       2021                     2134                105527     107661    2134  

[492 rows x 6 columns]

In [18]:
output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\population\QA_2021_01_sra_population.csv', index=True)
output

Group Quarters - College  Group Quarters - Military  \
sra    yr_id                                                        
Alpine 2010                          0                          0   
       2011                          0                          0   
       2012                          0                          0   
       2013                          0                          0   
       2014                          0                          0   
...                                ...                        ...   
Vista  2017                          0                          0   
       2018                          0                          0   
       2019                          0                          0   
       2020                          0                          0   
       2021                          0                          0   

              Group Quarters - Other  Household Population  total_pop  GQ_pop  
sra    yr_id                                                                   
Alpine 2010                      138                 15135      15273     138  
       2011                      103                 15406      15509     103  
       2012                      105                 15599      15704     105  
       2013                       88                 15794      15882      88  
       2014                      119                 15872      15991     119  
...                              ...                   ...        ...     ...  
Vista  2017                     2271                104503     106774    2271  
       2018                     2359                104642     107001    2359  
       2019                     2344                104568     106912    2344  
       2020                     2360                104649     107009    2360  
       2021                     2134                105527     107661    2134  

[492 rows x 6 columns]